# Notebook to create new geodatabase to store GDEs with stress/threat values

Build in metadata and field names/aliases.

Also copy to shapefiles for individual layer distribution.

## Set up environments in ArcGIS Pro

In [ ]:
# Import ArcGIS modules and check out spatial analyst extension
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

In [ ]:
# Path to temporary geodatabase
#path =  r"folder_path\gdbname.gdb"
path =  r"K:\GIS3\Projects\GDE\Maps\GDE_Threats\GDE_Threats.gdb"

# Environment settings
env.workspace = path
env.overwriteOutput = True
env.outputCoordinateSystem = arcpy.SpatialReference(26911) # Spatial reference NAD 1983 UTM Zone 11N. The code is '26911'

## Create a geodatabase

Here, copied the original iGDE database since we'll need all of layers in it and want to retain as many field aliases and metadata as possible. One could create a geodatabase from scratch and add all of the layers to it instead.

In [ ]:
og_gde = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_022120.gdb'
arcpy.Copy_management(og_gde, r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb')

In [ ]:
# iGDE geodatabase (copied version)
gdes = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb'
print(arcpy.ListFeatureClasses(gdes))

# database with stress/threat values
stats = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess2.gdb'
print(arcpy.ListFeatureClasses(stats))

### Springs

Methods used in Springs is nearly identical to all other layers. There will be fewer notes in the other sections

In [ ]:
# Copy stats to table
# Join doesn't like pulling right from the fc
# Can join from table instead, using OBJECTID fields; they're identical between table and feature class
tbl = arcpy.TableToTable_conversion(spr_st, env.workspace, 'spring_stats_tbl')


In [ ]:
# before join, change field order
fields_nowgt = ['Falling_GW', 'PumpStr', 'WW_Norm_Str', 'Shallow_GW', 'CommitThr', 'WW_Norm_Thr', 
          'Recharge', 'RenormThr', 
          'GRAZE', 'HORSE_BURRO', 'ELK',
          'Ung_Wgt_Str', 'Ung_Wgt_Thr', 
          'NN_Wgt_Str', 'NN_Wgt_Thr',
          'HousingStr', 'SurfaceDiv', 'HA_Wgt_Str', 'HA_Wgt_Thr',
          'ScoreStr', 'ScoreThr']

In [ ]:
# Join fields from table to copied gdb
arcpy.JoinField_management(spr_og, 'SPRING_ID', tbl, 'SPRING_ID', fields_nowgt)
[f.name for f in arcpy.ListFields(spr_og)]

In [ ]:
# Alter fields - update field alias an maybe field name
# Can use for other feature classes

fc = spr_og
arcpy.AlterField_management(fc, 'Falling_GW', 'Falling_GW', 'Falling Groundwater Level Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'PumpStr', 'Pump_GW', 'Pumping Status Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'WW_Norm_Str', 'GW_Str', 'Groundwater Withdrawal Stress Score')
arcpy.AlterField_management(fc, 'Shallow_GW', 'Shallow_GW', 'Shallow Groundwater Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'CommitThr', 'Commit_GW', 'Appropriation Status Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'WW_Norm_Thr', 'GW_Thr', 'Groundwater Withdrawal Threat Score')

arcpy.AlterField_management(fc, 'Recharge', 'Clim_Str', 'Climate Stress Score')
arcpy.AlterField_management(fc, 'RenormThr', 'Clim_Thr', 'Climate Threat Score')

arcpy.AlterField_management(fc, 'GRAZE', 'Ung_Graze', 'Cattle and Sheep Potential')
arcpy.AlterField_management(fc, 'HORSE_BURRO', 'Ung_WHB', 'Wild Horse and Burro Potential')
arcpy.AlterField_management(fc, 'ELK', 'Ung_Elk', 'Elk Potential')
arcpy.AlterField_management(fc, 'Ung_Wgt_Str', 'Ung_Str', 'Ungulate Stress Score')
arcpy.AlterField_management(fc, 'Ung_Wgt_Thr', 'Ung_Thr', 'Ungulate Threat Score')

arcpy.AlterField_management(fc, 'NN_Wgt_Str', 'NN_Str', 'Non-native Species Stress Score')
arcpy.AlterField_management(fc, 'NN_Wgt_Thr', 'NN_Thr', 'Non-native Species Threat Score')

arcpy.AlterField_management(fc, 'HousingStr', 'Hous_OHD', 'Housing Density Stress Score (other human development stress)')
arcpy.AlterField_management(fc, 'SurfaceDiv', 'SurfDiv_OHD', 'Surface Water Diversions (other human development stress)')
arcpy.AlterField_management(fc, 'HA_Wgt_Str', 'OHD_Str', 'Other Human Development Stress Score')
arcpy.AlterField_management(fc, 'HA_Wgt_Thr', 'OHD_Thr', 'Other Human Development Threat Score')

arcpy.AlterField_management(fc, 'ScoreStr', 'Score_Str', 'Overall Stress Score')
arcpy.AlterField_management(fc, 'ScoreThr', 'Score_Thr', 'Overall Threat Score')

[f.name for f in arcpy.ListFields(fc)]
# Need to manually move fields in ArcGIS Pro to change order in which they appear (optional)

### Wetlands

In [ ]:
wet_og = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\wetlands'
print(arcpy.GetCount_management(wet_og))
print([f.name for f in arcpy.ListFields(wet_og)])
wet_st = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess2.gdb\wetlands'
print(arcpy.GetCount_management(wet_st))
print([f.name for f in arcpy.ListFields(wet_st)])

In [ ]:
# Copy stats to table
tbl = arcpy.TableToTable_conversion(wet_st, env.workspace, 'wetland_stats_tbl')

fields_nowgt = ['Falling_GW', 'PumpStr', 'WW_Norm_Str', 'Shallow_GW', 'CommitThr', 'WW_Norm_Thr', 
          'Recharge', 'RenormThr', 
          'GRAZE', 'HORSE_BURRO', 'ELK',
          'Ung_Wgt_Str', 'Ung_Wgt_Thr', 
          'NN_Wgt_Str', 'NN_Wgt_Thr',
          'HousingStr', 'SurfaceDiv', 'HA_Wgt_Str', 'HA_Wgt_Thr',
          'ScoreStr', 'ScoreThr']

In [ ]:
arcpy.JoinField_management(wet_og, 'OBJECTID', tbl, 'OBJECTID', fields_nowgt)
print([f.name for f in arcpy.ListFields(wet_og)])

In [ ]:
fc = wet_og
arcpy.AlterField_management(fc, 'Falling_GW', 'Falling_GW', 'Falling Groundwater Level Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'PumpStr', 'Pump_GW', 'Pumping Status Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'WW_Norm_Str', 'GW_Str', 'Groundwater Withdrawal Stress Score')

arcpy.AlterField_management(fc, 'Shallow_GW', 'Shallow_GW', 'Shallow Groundwater Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'CommitThr', 'Commit_GW', 'Appropriation Status Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'WW_Norm_Thr', 'GW_Thr', 'Groundwater Withdrawal Threat Score')

arcpy.AlterField_management(fc, 'Recharge', 'Clim_Str', 'Climate Stress Score')
arcpy.AlterField_management(fc, 'RenormThr', 'Clim_Thr', 'Climate Threat Score')

arcpy.AlterField_management(fc, 'GRAZE', 'Ung_Graze', 'Cattle and Sheep Potential')
arcpy.AlterField_management(fc, 'HORSE_BURRO', 'Ung_WHB', 'Wild Horse and Burro Potential')
arcpy.AlterField_management(fc, 'ELK', 'Ung_Elk', 'Elk Potential')
arcpy.AlterField_management(fc, 'Ung_Wgt_Str', 'Ung_Str', 'Ungulate Stress Score')
arcpy.AlterField_management(fc, 'Ung_Wgt_Thr', 'Ung_Thr', 'Ungulate Threat Score')

arcpy.AlterField_management(fc, 'NN_Wgt_Str', 'NN_Str', 'Non-native Species Stress Score')
arcpy.AlterField_management(fc, 'NN_Wgt_Thr', 'NN_Thr', 'Non-native Species Threat Score')

arcpy.AlterField_management(fc, 'HousingStr', 'Hous_OHD', 'Housing Density Stress Score (other human development stress)')
arcpy.AlterField_management(fc, 'SurfaceDiv', 'SurfDiv_OHD', 'Surface Water Diversions (other human development stress)')
arcpy.AlterField_management(fc, 'HA_Wgt_Str', 'OHD_Str', 'Other Human Development Stress Score')
arcpy.AlterField_management(fc, 'HA_Wgt_Thr', 'OHD_Thr', 'Other Human Development Threat Score')

arcpy.AlterField_management(fc, 'ScoreStr', 'Score_Str', 'Overall Stress Score')
arcpy.AlterField_management(fc, 'ScoreThr', 'Score_Thr', 'Overall Threat Score')

print([f.name for f in arcpy.ListFields(fc)])

### Rivers & Streams

In [ ]:
rs_og = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\Rivers_Streams'
print(arcpy.GetCount_management(rs_og))
print([f.name for f in arcpy.ListFields(rs_og)])
rs_st = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess2.gdb\Rivers_Streams'
print(arcpy.GetCount_management(rs_st))
print([f.name for f in arcpy.ListFields(rs_st)])

In [ ]:
# Copy stats to table
# Join doesn't like drawing right from the fc
tbl = arcpy.TableToTable_conversion(rs_st, env.workspace, 'riverstreams_stats_tbl')

fields_nowgt = ['Falling_GW', 'PumpStr', 'WW_Norm_Str', 'Shallow_GW', 'CommitThr', 'WW_Norm_Thr', 
          'Recharge', 'RenormThr', 
          'GRAZE', 'HORSE_BURRO', 'ELK',
          'Ung_Wgt_Str', 'Ung_Wgt_Thr', 
          'NN_Wgt_Str', 'NN_Wgt_Thr',
          'HousingStr', 'SurfaceDiv', 'HA_Wgt_Str', 'HA_Wgt_Thr',
          'ScoreStr', 'ScoreThr']

arcpy.JoinField_management(rs_og, 'OBJECTID', tbl, 'OBJECTID', fields_nowgt)
print([f.name for f in arcpy.ListFields(rs_og)])

In [ ]:
fc = rs_og
arcpy.AlterField_management(fc, 'Falling_GW', 'Falling_GW', 'Falling Groundwater Level Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'PumpStr', 'Pump_GW', 'Pumping Status Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'WW_Norm_Str', 'GW_Str', 'Groundwater Withdrawal Stress Score')

arcpy.AlterField_management(fc, 'Shallow_GW', 'Shallow_GW', 'Shallow Groundwater Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'CommitThr', 'Commit_GW', 'Appropriation Status Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'WW_Norm_Thr', 'GW_Thr', 'Groundwater Withdrawal Threat Score')

arcpy.AlterField_management(fc, 'Recharge', 'Clim_Str', 'Climate Stress Score')
arcpy.AlterField_management(fc, 'RenormThr', 'Clim_Thr', 'Climate Threat Score')

arcpy.AlterField_management(fc, 'GRAZE', 'Ung_Graze', 'Cattle and Sheep Potential')
arcpy.AlterField_management(fc, 'HORSE_BURRO', 'Ung_WHB', 'Wild Horse and Burro Potential')
arcpy.AlterField_management(fc, 'ELK', 'Ung_Elk', 'Elk Potential')
arcpy.AlterField_management(fc, 'Ung_Wgt_Str', 'Ung_Str', 'Ungulate Stress Score')
arcpy.AlterField_management(fc, 'Ung_Wgt_Thr', 'Ung_Thr', 'Ungulate Threat Score')

arcpy.AlterField_management(fc, 'NN_Wgt_Str', 'NN_Str', 'Non-native Species Stress Score')
arcpy.AlterField_management(fc, 'NN_Wgt_Thr', 'NN_Thr', 'Non-native Species Threat Score')

arcpy.AlterField_management(fc, 'HousingStr', 'Hous_OHD', 'Housing Density Stress Score (other human development stress)')
arcpy.AlterField_management(fc, 'SurfaceDiv', 'SurfDiv_OHD', 'Surface Water Diversions (other human development stress)')
arcpy.AlterField_management(fc, 'HA_Wgt_Str', 'OHD_Str', 'Other Human Development Stress Score')
arcpy.AlterField_management(fc, 'HA_Wgt_Thr', 'OHD_Thr', 'Other Human Development Threat Score')

arcpy.AlterField_management(fc, 'ScoreStr', 'Score_Str', 'Overall Stress Score')
arcpy.AlterField_management(fc, 'ScoreThr', 'Score_Thr', 'Overall Threat Score')

print([f.name for f in arcpy.ListFields(fc)])


### Lakes & Playas

In [ ]:
lp_og = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\Lakes_Playas'
print(arcpy.GetCount_management(lp_og))
print([f.name for f in arcpy.ListFields(lp_og)])
lp_st = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess2.gdb\Lakes_Playas'
print(arcpy.GetCount_management(lp_st))
print([f.name for f in arcpy.ListFields(lp_st)])

In [ ]:
# Copy stats to table
tbl = arcpy.TableToTable_conversion(lp_st, env.workspace, 'lakesplayas_stats_tbl')

# To match other feature classes, add intermediate ungulate fields
# All values == 0 (no ungulate stress/threat to lakes & playas)
arcpy.AddField_management(tbl, 'GRAZE', 'LONG')
arcpy.AddField_management(tbl, 'HORSE_BURRO', 'LONG')
arcpy.AddField_management(tbl, 'ELK', 'LONG')
ung_fields = ['GRAZE', 'HORSE_BURRO', 'ELK']
arcpy.CalculateField_management(tbl, ung_fields[0], 0, 'PYTHON3')
arcpy.CalculateField_management(tbl, ung_fields[1], 0, 'PYTHON3')
arcpy.CalculateField_management(tbl, ung_fields[2], 0, 'PYTHON3')

In [ ]:
fields_nowgt = ['Falling_GW', 'PumpStr', 'WW_Norm_Str', 'Shallow_GW', 'CommitThr', 'WW_Norm_Thr', 
          'Recharge', 'RenormThr', 
          'GRAZE', 'HORSE_BURRO', 'ELK',
          'Ung_Wgt_Str', 'Ung_Wgt_Thr', 
          'NN_Wgt_Str', 'NN_Wgt_Thr',
          'HousingStr', 'SurfaceDiv', 'HA_Wgt_Str', 'HA_Wgt_Thr',
          'ScoreStr', 'ScoreThr']


arcpy.JoinField_management(lp_og, 'OBJECTID', tbl, 'OBJECTID', fields_nowgt)
print([f.name for f in arcpy.ListFields(lp_og)])

In [ ]:
fc = lp_og
arcpy.AlterField_management(fc, 'Falling_GW', 'Falling_GW', 'Falling Groundwater Level Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'PumpStr', 'Pump_GW', 'Pumping Status Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'WW_Norm_Str', 'GW_Str', 'Groundwater Withdrawal Stress Score')

arcpy.AlterField_management(fc, 'Shallow_GW', 'Shallow_GW', 'Shallow Groundwater Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'CommitThr', 'Commit_GW', 'Appropriation Status Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'WW_Norm_Thr', 'GW_Thr', 'Groundwater Withdrawal Threat Score')

arcpy.AlterField_management(fc, 'Recharge', 'Clim_Str', 'Climate Stress Score')
arcpy.AlterField_management(fc, 'RenormThr', 'Clim_Thr', 'Climate Threat Score')

arcpy.AlterField_management(fc, 'GRAZE', 'Ung_Graze', 'Cattle and Sheep Potential')
arcpy.AlterField_management(fc, 'HORSE_BURRO', 'Ung_WHB', 'Wild Horse and Burro Potential')
arcpy.AlterField_management(fc, 'ELK', 'Ung_Elk', 'Elk Potential')
arcpy.AlterField_management(fc, 'Ung_Wgt_Str', 'Ung_Str', 'Ungulate Stress Score')
arcpy.AlterField_management(fc, 'Ung_Wgt_Thr', 'Ung_Thr', 'Ungulate Threat Score')

arcpy.AlterField_management(fc, 'NN_Wgt_Str', 'NN_Str', 'Non-native Species Stress Score')
arcpy.AlterField_management(fc, 'NN_Wgt_Thr', 'NN_Thr', 'Non-native Species Threat Score')

arcpy.AlterField_management(fc, 'HousingStr', 'Hous_OHD', 'Housing Density Stress Score (other human development stress)')
arcpy.AlterField_management(fc, 'SurfaceDiv', 'SurfDiv_OHD', 'Surface Water Diversions (other human development stress)')
arcpy.AlterField_management(fc, 'HA_Wgt_Str', 'OHD_Str', 'Other Human Development Stress Score')
arcpy.AlterField_management(fc, 'HA_Wgt_Thr', 'OHD_Thr', 'Other Human Development Threat Score')

arcpy.AlterField_management(fc, 'ScoreStr', 'Score_Str', 'Overall Stress Score')
arcpy.AlterField_management(fc, 'ScoreThr', 'Score_Thr', 'Overall Threat Score')

print([f.name for f in arcpy.ListFields(fc)])

### Phreatophytes

Slightly different method than other feature classes; since we "exploded" (multipart to singlepart) the phreatophytes layer to better calculate stress/threat to individual phreatophyte features, we'll just replace the copied layer in the geodatabase with the one in the stress/threats stats geodatabase.

In [ ]:
phr_og = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\Phreatophytes'
print(arcpy.GetCount_management(phr_og))
print([f.name for f in arcpy.ListFields(phr_og)])
phr_st = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess2.gdb\Phreatophytes_Explode'
print(arcpy.GetCount_management(phr_st))
print([f.name for f in arcpy.ListFields(phr_st)])

In [ ]:
# Delete Phreatophytes feature in new GDB
arcpy.Delete_management(phr_og)

In [ ]:
# Copy exploded phreatophytes to new GDB
# Only copy over necessary fields
phr_fields = ['SOURCE_CODE', 'PHR_TYPE', 'PHR_GROUP', 'COMMENTS',
            'Falling_GW', 'PumpStr', 'WW_Norm_Str', 'Shallow_GW', 'CommitThr', 'WW_Norm_Thr', 
            'Recharge', 'RenormThr', 
            'GRAZE', 'HORSE_BURR', 'ELK',
            'Ung_Wgt_Str', 'Ung_Wgt_Thr', 
            'NN_Wgt_Str', 'NN_Wgt_Thr',
            'HousingStr', 'SurfaceDiv', 'HA_Wgt_Str', 'HA_Wgt_Thr',
            'ScoreStr', 'ScoreThr']

In [ ]:
# Field mapping object
fmap = arcpy.FieldMappings()
for f in phr_fields:
    print(f)
    fm = arcpy.FieldMap()
    fm.addInputField(phr_st, f)
    fmap.addFieldMap(fm)
    
# Copy to GDB with only named fields - will take a minute
arcpy.FeatureClassToFeatureClass_conversion(phr_st, r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb', 'Phreatophytes',
                                            '', fmap)
print([f.name for f in arcpy.ListFields(phr_st)])

In [ ]:
phr_new = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\Phreatophytes'
fc = phr_new
arcpy.AlterField_management(fc, 'Falling_GW', 'Falling_GW', 'Falling Groundwater Level Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'PumpStr', 'Pump_GW', 'Pumping Status Score (groundwater withdrawal stress)')
arcpy.AlterField_management(fc, 'WW_Norm_Str', 'GW_Str', 'Groundwater Withdrawal Stress Score')

arcpy.AlterField_management(fc, 'Shallow_GW', 'Shallow_GW', 'Shallow Groundwater Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'CommitThr', 'Commit_GW', 'Appropriation Status Score (groundwater withdrawal threat)')
arcpy.AlterField_management(fc, 'WW_Norm_Thr', 'GW_Thr', 'Groundwater Withdrawal Threat Score')

arcpy.AlterField_management(fc, 'Recharge', 'Clim_Str', 'Climate Stress Score')
arcpy.AlterField_management(fc, 'RenormThr', 'Clim_Thr', 'Climate Threat Score')

arcpy.AlterField_management(fc, 'GRAZE', 'Ung_Graze', 'Cattle and Sheep Potential')
arcpy.AlterField_management(fc, 'HORSE_BURR', 'Ung_WHB', 'Wild Horse and Burro Potential')
arcpy.AlterField_management(fc, 'ELK', 'Ung_Elk', 'Elk Potential')
arcpy.AlterField_management(fc, 'Ung_Wgt_Str', 'Ung_Str', 'Ungulate Stress Score')
arcpy.AlterField_management(fc, 'Ung_Wgt_Thr', 'Ung_Thr', 'Ungulate Threat Score')

arcpy.AlterField_management(fc, 'NN_Wgt_Str', 'NN_Str', 'Non-native Species Stress Score')
arcpy.AlterField_management(fc, 'NN_Wgt_Thr', 'NN_Thr', 'Non-native Species Threat Score')

arcpy.AlterField_management(fc, 'HousingStr', 'Hous_OHD', 'Housing Density Stress Score (other human development stress)')
arcpy.AlterField_management(fc, 'SurfaceDiv', 'SurfDiv_OHD', 'Surface Water Diversions (other human development stress)')
arcpy.AlterField_management(fc, 'HA_Wgt_Str', 'OHD_Str', 'Other Human Development Stress Score')
arcpy.AlterField_management(fc, 'HA_Wgt_Thr', 'OHD_Thr', 'Other Human Development Threat Score')

arcpy.AlterField_management(fc, 'ScoreStr', 'Score_Str', 'Overall Stress Score')
arcpy.AlterField_management(fc, 'ScoreThr', 'Score_Thr', 'Overall Threat Score')

print([f.name for f in arcpy.ListFields(fc)])

## Copy to shapefiles

Or any non-geodatabase format

In [ ]:
# Folder to write shapefiles to
#shps = r'shp_folder_path'
shps = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps'

# Geodatabase to pull from
#gdb_path = r'path_to_folder\gdbname.gdb'
gdb_path = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb'
env.workspace = gdb_path
print(arcpy.ListFeatures(gdes))

# # Feature classes with stress/threat stats
# arcpy.CopyFeatures_management(spr_og, r'shp_folder_path\NV_iGDE_Springs.shp')
# arcpy.CopyFeatures_management(wet_og, r'shp_folder_path\NV_iGDE_Wetlands.shp')
# arcpy.CopyFeatures_management(rs_og, r'shp_folder_path\NV_iGDE_Rivers_Streams.shp')
# arcpy.CopyFeatures_management(lp_og, r'shp_folder_path\NV_iGDE_Lakes_Playas.shp')
# arcpy.CopyFeatures_management(phr_og, r'shp_folder_path\NV_iGDE_Phreatohpytes.shp')

# # Tables (optional) from of the geodatabase
# arcpy.CopyFeatures_management('path_to_folder\gdbname.gdb\Species', 
#                               r'shp_folder_path\NV_iGDE_Species.shp')
# arcpy.TableToTable_conversion('path_to_folder\gdbname.gdb\Species_tbl', 
#                               r'shp_folder_path', 'NV_iGDE_Species_tbl.dbf')
# arcpy.TableToTable_conversion('path_to_folder\gdbname.gdb\Source_tbl', 
#                               r'shp_folder_path', 'NV_iGDE_Source_tbl.dbf')


arcpy.CopyFeatures_management(spr_og, r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps\NV_iGDE_Springs.shp')
arcpy.CopyFeatures_management(wet_og, r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps\NV_iGDE_Wetlands.shp')
arcpy.CopyFeatures_management(rs_og, r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps\NV_iGDE_Rivers_Streams.shp')
arcpy.CopyFeatures_management(lp_og, r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps\NV_iGDE_Lakes_Playas.shp')
arcpy.CopyFeatures_management(phr_og, r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps\NV_iGDE_Phreatohpytes.shp')

arcpy.CopyFeatures_management('K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\Species', 
                              r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps\NV_iGDE_Species.shp')
arcpy.TableToTable_conversion('K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\Species_tbl', 
                              r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps', 'NV_iGDE_Species_tbl.dbf')
arcpy.TableToTable_conversion('K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322.gdb\Source_tbl', 
                              r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_032322_shps', 'NV_iGDE_Source_tbl.dbf')
